<a href="https://colab.research.google.com/github/spatank/CIS-530/blob/master/Homework%207/homework_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [26]:
# from nltk.corpus import conll2002
import nltk 
nltk.download('conll2002')
from nltk.corpus import conll2002
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Perceptron, RidgeClassifier, PassiveAggressiveClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import precision_recall_fscore_support
import pickle

# Assignment 7: NER
# Rebecca Iglesias-Flores and Shubhankar Patankar
# This is just to help you get going. Feel free to
# add to or modify any part of it.

[nltk_data] Downloading package conll2002 to /root/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


In [0]:
def hasDot(word):
  return int('.' in word)

def hasApost(word):
  return int('\'' in word)

def hasHyph(word):
  return int('-' in word)

def hasNC(pos):
  return int('NC' in pos)

def hasAQ(pos):
  return int('AQ' in pos)

def isCap(word):
    ## if first letter of the word is capitalized
  return int(word[0].isupper()) 

def hasCap(word):
    ## if any letter of the word is capitalized or not
  return int(word.islower())

accents = 'ÂÃÄÀÁÅÆÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖØÙÚÛÜÝÞßàáâãäåæçèéêëìíîïðñòóôõöøùúûüýþÿ'

def hasAcc(word):
  for char in word:
    if char in accents:
      return int(True)
  return int(False)

def hasDig(word):
  ## assigns a 1 if there's a digit in the word
  return int(word.isalpha())

def prefix(word):
  return word[:4]

def suffix(word):
  return word[-4:]

def wordshape(word):
    import re
    t1 = re.sub('[A-Z]', 'X', word)
    t2 = re.sub('[a-z]', 'x', t1)
    return re.sub('[0-9]', 'd', t2)

def getfeats(word, pos_tag, o):
  """ This takes the word in question and
  the offset with respect to the instance
  word """
  o = str(o)
  features = [
              (o + 'word', word),
              (o + 'shape', wordshape(word)),
              # TODO: add more features here.
              # (o + 'hasDot', hasDot(word)),
              (o + 'hasApost', hasApost(word)),
              (o + 'hasHyph', hasHyph(word)),
              (o + 'hasNC', hasNC(pos_tag)),
              (o + 'hasAQ', hasAQ(pos_tag)),
              (o + 'isCap', isCap(word)),
              (o + 'hasCap', hasCap(word)),
              (o + 'hasAcc', hasAcc(word)),
              # (o + 'hasDig', hasDig(word)),
              (o + 'prefix', prefix(word)),
              (o + 'suffix', suffix(word))
              ]
  return features
    
def word2features(sent, i):
  """ The function generates all features
  for the word at position i in the
  sentence."""
  features = []
  # the window around the token
  for o in [-3,-2,-1,0,1,2,3]:
    if i+o >= 0 and i+o < len(sent):
      word = sent[i+o][0]
      pos_tag = sent[i+o][1]
      featlist = getfeats(word, pos_tag, o)
      features.extend(featlist)
  
  return dict(features)

In [0]:
train_sents = list(conll2002.iob_sents('esp.train'))
dev_sents = list(conll2002.iob_sents('esp.testa'))
test_sents = list(conll2002.iob_sents('esp.testb'))

In [77]:
dev_sents[108]

[('(', 'Fpa', 'O'),
 ('URGENCIAS', 'NC', 'O'),
 ('061', 'Z', 'O'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

In [0]:
# train_feats = []
# train_labels = []

# for sent in train_sents:
#   for i in range(len(sent)):
#     feats = word2features(sent, i)
#     train_feats.append(feats)
#     train_labels.append(sent[i][-1])

# vectorizer = DictVectorizer()
# X_train = vectorizer.fit_transform(train_feats)

In [138]:
if __name__ == "__main__":

  train_sents = list(conll2002.iob_sents('esp.train'))
  train_feats = []
  train_labels = []
  for sent in train_sents:
    for i in range(len(sent)):
      feats = word2features(sent,i)
      train_feats.append(feats)
      train_labels.append(sent[i][-1])

  vectorizer = DictVectorizer()
  X_train = vectorizer.fit_transform(train_feats)

  # model = Perceptron(verbose = 1, max_iter = 2000)
  # model = RidgeClassifier()
  model = PassiveAggressiveClassifier(verbose = 1, loss = 'squared_hinge')
  # model = SGDClassifier(verbose = 1)
  model.fit(X_train, train_labels)
  pickle.dump(model, open('drive/My Drive/CIS-530/Homework 7/Results/model', 'wb'))

  # TRAINING SET
  y_pred_train = model.predict(X_train)  
  j = 0
  print("Writing to train_results.txt")
  # format is: word gold pred
  outfile_path = 'drive/My Drive/CIS-530/Homework 7/Results/train_results.txt' 
  with open(outfile_path, "w") as out:
    for sent in train_sents: 
      for i in range(len(sent)):
        word = sent[i][0]
        gold = sent[i][-1]
        pred = y_pred_train[j]
        j += 1
        out.write("{}\t{}\t{}\n".format(word, gold, pred))
    out.write("\n")

  # DEVELOPMENT SET
  dev_sents = list(conll2002.iob_sents('esp.testa'))
  dev_feats = []
  dev_labels = []
  for sent in dev_sents:
    for i in range(len(sent)):
      feats = word2features(sent,i)
      dev_feats.append(feats)
      dev_labels.append(sent[i][-1])
  X_dev = vectorizer.transform(dev_feats)
  y_pred_dev = model.predict(X_dev)
  j = 0
  print("Writing to dev_results.txt")
  # format is: word gold pred
  outfile_path = 'drive/My Drive/CIS-530/Homework 7/Results/dev_results.txt'
  with open(outfile_path, "w") as out:
    for sent in dev_sents: 
      for i in range(len(sent)):
        word = sent[i][0]
        gold = sent[i][-1]
        pred = y_pred_dev[j]
        j += 1
        out.write("{}\t{}\t{}\n".format(word,gold,pred))
    out.write("\n")

  # TEST SET
  test_sents = list(conll2002.iob_sents('esp.testb'))
  test_feats = []
  test_labels = []
  for sent in test_sents:
    for i in range(len(sent)):
      feats = word2features(sent,i)
      test_feats.append(feats)
      test_labels.append(sent[i][-1])
  X_test = vectorizer.transform(test_feats)
  y_pred_test = model.predict(X_test)
  j = 0
  print("Writing to test_results.txt")
  # format is: word gold pred
  outfile_path = 'drive/My Drive/CIS-530/Homework 7/Results/test_results.txt'
  with open(outfile_path, "w") as out:
    for sent in test_sents: 
      for i in range(len(sent)):
        word = sent[i][0]
        gold = sent[i][-1]
        pred = y_pred_test[j]
        j += 1
        out.write("{}\t{}\t{}\n".format(word,gold,pred))
    out.write("\n")

  print("Now run: python conlleval.py test_results.txt")

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-- Epoch 1
Norm: 13.37, NNZs: 61142, Bias: -0.083439, T: 264715, Avg. loss: 0.032485
Total training time: 0.18 seconds.
-- Epoch 2
Norm: 17.53, NNZs: 69338, Bias: -0.097949, T: 529430, Avg. loss: 0.018526
Total training time: 0.34 seconds.
-- Epoch 3
Norm: 20.37, NNZs: 72538, Bias: -0.103246, T: 794145, Avg. loss: 0.014014
Total training time: 0.49 seconds.
-- Epoch 4
Norm: 22.58, NNZs: 74295, Bias: -0.107065, T: 1058860, Avg. loss: 0.011057
Total training time: 0.64 seconds.
-- Epoch 5
Norm: 24.31, NNZs: 75360, Bias: -0.111805, T: 1323575, Avg. loss: 0.009125
Total training time: 0.79 seconds.
-- Epoch 6
Norm: 25.77, NNZs: 75922, Bias: -0.113951, T: 1588290, Avg. loss: 0.007848
Total training time: 0.94 seconds.
-- Epoch 7
Norm: 27.00, NNZs: 76250, Bias: -0.115219, T: 1853005, Avg. loss: 0.006821
Total training time: 1.10 seconds.
-- Epoch 8
Norm: 28.04, NNZs: 76498, Bias: -0.116651, T: 2117720, Avg. loss: 0.005911
Total training time: 1.25 seconds.
-- Epoch 9
Norm: 28.97, NNZs: 76704

[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   15.9s finished


Writing to train_results.txt
Writing to dev_results.txt
Writing to test_results.txt
Now run: python conlleval.py test_results.txt


In [139]:
!python 'drive/My Drive/CIS-530/Homework 7/conlleval.py' 'drive/My Drive/CIS-530/Homework 7/Results/train_results.txt'

processed 264715 tokens with 18797 phrases; found: 18934 phrases; correct: 18583.
accuracy:  99.88%; precision:  98.15%; recall:  98.86%; FB1:  98.50
              LOC: precision:  97.86%; recall:  98.72%; FB1:  98.29  4956
             MISC: precision:  96.41%; recall:  97.70%; FB1:  97.05  2202
              ORG: precision:  97.97%; recall:  98.73%; FB1:  98.35  7447
              PER: precision:  99.65%; recall:  99.84%; FB1:  99.75  4329


In [140]:
!python 'drive/My Drive/CIS-530/Homework 7/conlleval.py' 'drive/My Drive/CIS-530/Homework 7/Results/dev_results.txt'

processed 52923 tokens with 4351 phrases; found: 4633 phrases; correct: 3097.
accuracy:  95.90%; precision:  66.85%; recall:  71.18%; FB1:  68.94
              LOC: precision:  62.92%; recall:  77.95%; FB1:  69.63  1219
             MISC: precision:  39.14%; recall:  40.90%; FB1:  40.00  465
              ORG: precision:  69.01%; recall:  69.18%; FB1:  69.10  1704
              PER: precision:  78.07%; recall:  79.54%; FB1:  78.80  1245


In [141]:
!python 'drive/My Drive/CIS-530/Homework 7/conlleval.py' 'drive/My Drive/CIS-530/Homework 7/Results/test_results.txt'

processed 51533 tokens with 3558 phrases; found: 3866 phrases; correct: 2639.
accuracy:  96.89%; precision:  68.26%; recall:  74.17%; FB1:  71.09
              LOC: precision:  75.61%; recall:  72.05%; FB1:  73.78  1033
             MISC: precision:  38.14%; recall:  39.82%; FB1:  38.96  354
              ORG: precision:  66.79%; recall:  75.71%; FB1:  70.97  1587
              PER: precision:  74.33%; recall:  90.20%; FB1:  81.50  892
